In [0]:
#####################################################
                                                    
# written by Rija Tonny C. Ramarolahy               
                                                     
# 2020                                               
                                                    
# email: rija@aims.edu.gh / minuramaro@gmail.com   
                                                     
#####################################################


import tensorflow as tf
import numpy as np

#%load_ext tensorboard.notebook
tf.random.set_seed(42)

########## LOAD DATA #########
#if you use google colab with your drive.
train = np.load('/content/drive/My Drive/data/train.npz')
val = np.load('/content/drive/My Drive/data/val.npz')

train_imgs = train['img']
train_labels = train['label']
val_imgs = val['img']
val_labels = val['label']


In [0]:
#################################### CNN MODEL ##########################
#### Construnction of the model ########
tf.random.set_seed(42)
INPUT_SHAPE = (100, 100, 3)

inp = tf.keras.layers.Input(shape=INPUT_SHAPE)

conv1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', padding='same')(inp)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), 
                               activation='relu', padding='same')(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), 
                               activation='relu', padding='same')(pool2)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

flat = tf.keras.layers.Flatten()(pool3)

hidden1 = tf.keras.layers.Dense(512, activation='relu')(flat)
drop1 = tf.keras.layers.Dropout(rate=0.3)(hidden1)
hidden2 = tf.keras.layers.Dense(512, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(rate=0.3)(hidden2)

out = tf.keras.layers.Dense(1, activation='sigmoid')(drop2)

model = tf.keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
model.summary()




In [0]:

################################ TRAINING ###############################
import datetime 
import time
import os

start_time = time.time()
BATCH_SIZE = 64
EPOCHS = 25
#in case of using drive and colab
logdir = os.path.join('/content/drive/My Drive/callbacks', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)

                                             
callbacks = [reduce_lr, tensorboard_callback]

history = model.fit(x=train_imgs, y=train_labels, 
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS, 
                    validation_data=(val_imgs, val_labels), 
                    callbacks=callbacks,
                    verbose=1)

elapsed_time = time.time() - start_time
print("Time of the training")
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [0]:
############## PLOT ACCURACY #########

import matplotlib.pyplot as plt
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Basic CNN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

max_epoch = len(history.history['accuracy'])+1
epoch_list = list(range(1,max_epoch))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(1, max_epoch, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(1, max_epoch, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

###### save the model #####

#case of colab and drive
model.save('/content/drive/My Drive/models/basic_CNN_.h5')

In [0]:
############### TEST ##############
#compute the ROC-AUC values and plot ROC
from sklearn.metrics import roc_curve, auc 
from sklearn.model_selection import LeaveOneOut

test = np.load('/content/drive/My Drive/data/test.npz')
test_imgs = test['img']
test_labels = test['label']
#cross validation
loo = LeaveOneOut()

for train_index, test_index in loo.split(test_imgs, test_labels):
    test_im, drop_test = test_imgs[train_index], test_imgs[test_index]
    test_lab, drop_label = test_labels[train_index], test_labels[test_index]


#basic_cnn_model = tf.keras.models.load_model('/content/drive/My Drive/models/basic_CNN_.h5')

test_label_predict = model.predict(test_im, batch_size=512)
fpr, tpr, _ = roc_curve(test_lab, test_label_predict[:,0]) 
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'.format(roc_auc),linewidth=2.5)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [0]:
##### CONFUSION METRICS #######
from sklearn import metrics

test_label_predict = model.predict(test_imgs , batch_size=512)
test_label_predict[:] = [np.where(test_label_predict[i] > 0.5, 1, 0) for i in range(len(test_label_predict))]

def get_metrics(true_labels, predicted_labels): 
    cm = metrics.confusion_matrix(true_labels, predicted_labels)
    tp = cm[0,0]
    tn = cm[1,1]
    fp = cm[0,1]
    fn = cm[1,0]

    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    return {
        'Accuracy': np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4),
        'Precision': np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4),
        'Recall': np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4),
        'F1 Score': np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4),
        'Sensivity': np.round(sensitivity,4),
        'Specificity': np.round(specificity,3)
        }


import pandas as pd

model_metrics = get_metrics(true_labels= test_labels, predicted_labels=test_label_predict)
pd.DataFrame([model_metrics], index=['Basic CNN']) 